In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

!pip install --ignore-installed --user --upgrade tensorflow

In [2]:
tf.__version__

'2.10.1'

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu.name)
else:
    print("Failed to detect a GPU.")

Found a GPU with the name: /physical_device:GPU:0


In [4]:
data= pd.read_csv(r"data/fra.txt", delimiter='\t', header=None)

In [5]:
data= data.iloc[:,:-1]

In [6]:
data.head()

,0,1
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [7]:
data.shape

(229803, 2)

In [8]:
data.rename(columns= {0:'English', 1:'French'}, inplace= True)

In [9]:
data.head()

,English,French
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


In [10]:
data['English']= data['English'].astype(str)
data['French']= data['French'].astype(str)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229803 entries, 0 to 229802
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   English  229803 non-null  object
 1   French   229803 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [12]:
from string import punctuation
import re

def clean_string(string):
    # Replace no-break space with space
    string = string.replace("\u202f"," ")
    # Converts all uppercase characters into lowercase characters
    string = string.lower()

    # Delete the punctuation and the numbers
    for p in punctuation + "«»" + "0123456789":
        string = string.replace(p," ")

    # Eliminate duplicate whitespaces using wildcards
    string = re.sub("\s+"," ", string)
    # Remove spaces at the beginning and at the end of the string
    string = string.strip()

    return string

In [13]:
data['English'] = data['English'].apply(lambda x: clean_string(x))
data['French'] = data['French'].apply(lambda x: clean_string(x))

In [14]:
# data= data.iloc[:15000, :]

In [15]:
data.shape

(229803, 2)

In [16]:
data.head()

,English,French
0,go,va
1,go,marche
2,go,en route
3,go,bouge
4,hi,salut


In [17]:
source_sentences= data['English'].values
target_sentences= data['French'].values

In [18]:
train_data_split= int(source_sentences.shape[0] * 0.1 )
train_data_split

22980

In [19]:
source_sentences, source_val_sentences= source_sentences[train_data_split:], source_sentences[:train_data_split]
target_sentences, target_val_sentences= target_sentences[train_data_split:], target_sentences[:train_data_split]

In [20]:
def tag_target_sentences(sentences):
  tagged_sentences = map(lambda s: (' ').join(['<sos>', s, '<eos>']), sentences)
  return list(tagged_sentences)

In [21]:
target_sentences= tag_target_sentences(target_sentences)

In [22]:
target_sentences[:5]

['<sos> ils sont artistes <eos>',
 '<sos> elles sont artistes <eos>',
 '<sos> ils sont médecins <eos>',
 '<sos> elles sont médecins <eos>',
 '<sos> ils sont chanteurs <eos>']

In [23]:
len(target_sentences)

206823

### Tokenization

In [24]:
# SOURCE Tokenization
source_tokenizer= tf.keras.preprocessing.text.Tokenizer(oov_token="<unk>", filters="#$%&()*+,-./:;=@[\\]^_`{|}~\t\n")
source_tokenizer.fit_on_texts(source_sentences)

In [25]:
train_encoder_inputs = source_tokenizer.texts_to_sequences(source_sentences)

In [26]:
source_vocab_size= len(source_tokenizer.word_index) + 1
source_vocab_size

15639

In [27]:
print(train_encoder_inputs[60:65])
print(source_tokenizer.sequences_to_texts(train_encoder_inputs[60:65]))

[[43, 58, 510], [43, 58, 626], [43, 58, 626], [43, 58, 626], [43, 58, 626]]
['they were hungry', 'they were killed', 'they were killed', 'they were killed', 'they were killed']


In [28]:
# TARGET Tokenization
target_tokenizer= tf.keras.preprocessing.text.Tokenizer(oov_token="<unk>", filters="#$%&()*+,-./:;=@[\\]^_`{|}~\t\n")
target_tokenizer.fit_on_texts(target_sentences)

In [29]:
target_vocab_size= len(target_tokenizer.word_index) + 1
target_vocab_size

26368

In [30]:
def generate_decoder_inputs_targets(sequences, tokenizer):
    seqs= tokenizer.texts_to_sequences(sequences)
    decoder_inputs= [s[:-1] for s in seqs] # Drops last token in sequence; eg = <sos> Hi I am Arvind
    decoder_outputs= [s[1:] for s in seqs] # Drops first token in sequence; eg = Hi I am Arvind <eos>

    return decoder_inputs, decoder_outputs

In [31]:
train_decoder_inputs, train_decoder_outputs= generate_decoder_inputs_targets(target_sentences, target_tokenizer)

In [32]:
# decoder inputs 
target_tokenizer.sequences_to_texts(train_decoder_inputs[:5])

['<sos> ils sont artistes',
 '<sos> elles sont artistes',
 '<sos> ils sont médecins',
 '<sos> elles sont médecins',
 '<sos> ils sont chanteurs']

In [33]:
# decoder outputs
target_tokenizer.sequences_to_texts(train_decoder_outputs[:5])

['ils sont artistes <eos>',
 'elles sont artistes <eos>',
 'ils sont médecins <eos>',
 'elles sont médecins <eos>',
 'ils sont chanteurs <eos>']

In [34]:
max_encoding_len= len(max(train_encoder_inputs, key= len))
max_encoding_len

59

In [35]:
max_decoding_len= len(max(train_decoder_inputs, key= len))
max_decoding_len

63

### Padding

In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_train_encoder_inputs= pad_sequences(train_encoder_inputs, maxlen= max_encoding_len, padding= "post", truncating= "post")
padded_train_decoder_inputs= pad_sequences(train_decoder_inputs, maxlen= max_decoding_len, padding= "post", truncating= "post")
padded_train_decoder_outputs= pad_sequences(train_decoder_outputs, maxlen= max_decoding_len, padding= "post", truncating= "post")

In [37]:
print(padded_train_encoder_inputs[10])
print(padded_train_decoder_inputs[10])
print(padded_train_decoder_outputs[10])

[  43 3325   93    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]
[   2  110   24   92 7086    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]
[ 110   24   92 7086    3    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0]


In [38]:
# 0 is considered as OOV and given <unk> value
source_tokenizer.sequences_to_texts([padded_train_encoder_inputs[80000]])

['it s a pleasure to be here <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>']

### validation data preparation

In [39]:
def process_dataset(preprocessed_input, preprocessed_output):
    
    tagged_preprocessed_output = tag_target_sentences(preprocessed_output)
    # Vectorize encoder source sentences.
    encoder_inputs = source_tokenizer.texts_to_sequences(preprocessed_input)
    # Vectorize and create decoder input and target sentences.
    decoder_inputs, decoder_targets = generate_decoder_inputs_targets(tagged_preprocessed_output, 
                                                                    target_tokenizer)
  
    # Pad all collections.
    padded_encoder_inputs = pad_sequences(encoder_inputs, max_encoding_len, padding='post', truncating='post')
    padded_decoder_inputs = pad_sequences(decoder_inputs, max_decoding_len, padding='post', truncating='post')
    padded_decoder_targets = pad_sequences(decoder_targets, max_decoding_len, padding='post', truncating='post')

    return padded_encoder_inputs, padded_decoder_inputs, padded_decoder_targets


In [40]:
# Process validation dataset
padded_val_encoder_inputs, padded_val_decoder_inputs, padded_val_decoder_targets= process_dataset(source_val_sentences, target_val_sentences)

In [41]:
target_tokenizer.sequences_to_texts([padded_val_decoder_inputs[1780]])

['<sos> soyez satisfait <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>']

### Building Translation Model

In [42]:
embedding_dim= 128
hidden_dim= 256
default_dropout= 0.2
batch_size= 32
epochs= 30

In [66]:
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Dense

encoder_inputs= Input(shape= [None], name= 'encoder_inputs')
#  mask_zero: Boolean, whether or not the input value 0 is a special
encoder_embeddings= Embedding(input_dim= source_vocab_size, output_dim= embedding_dim, mask_zero= True, name= "encoder_embeddings")
encoder_embedding_outputs= encoder_embeddings(encoder_inputs)

encoder_lstm= LSTM(units= hidden_dim, return_state= True, dropout= default_dropout, name= "encoder_lstm")
# since return sequences is false: the value of encoder_outputs is same as state_h, 
# if true, all y_hat values of every time stamp is returned to encoder_outputs
encoder_outputs, state_h, state_c= encoder_lstm(encoder_embedding_outputs)

encoder_states= (state_h, state_c)


In [67]:
decoder_inputs= Input(shape= [None], name= "decoder_inputs")
decoder_embeddings= Embedding(input_dim= target_vocab_size, output_dim= embedding_dim, mask_zero= True, name= "decoder_embeddings")
decoder_embedding_outputs= decoder_embeddings(decoder_inputs)

decoder_lstm= LSTM(units= hidden_dim, return_sequences=True ,return_state= True, dropout= default_dropout, name= "decoder_lstm")

decoder_outputs, _, _= decoder_lstm(decoder_embedding_outputs, initial_state= encoder_states)

decoder_dense= Dense(target_vocab_size, activation='softmax', name="decoder_dense")

y_proba= decoder_dense(decoder_outputs)

In [45]:
# Note how the model is taking two inputs in an array.
model = tf.keras.Model([encoder_inputs, decoder_inputs], y_proba, name='eng_fre_seq2seq_nmt_no_attention')

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',  metrics=['accuracy'])
model.summary()

Model: "eng_fre_seq2seq_nmt_no_attention"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 encoder_embeddings (Embedding)  (None, None, 128)   2001792     ['encoder_inputs[0][0]']         
                                                                                                  
 decoder_embeddings (Embedding)  (None, None, 128)   3375104     ['decoder_inputs[0][0]']         
                                                                   

In [46]:
from keras.utils.vis_utils import plot_model

plot_model(model, to_file='eng_fre_seq2seq_nmt_no_attention.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [47]:
# Saving this to a folder on my local machine.
filepath="./EngFreNMTNoAttention/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

In [62]:
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit([padded_train_encoder_inputs, padded_train_decoder_inputs], padded_train_decoder_outputs,
                     batch_size=batch_size,
                     epochs=epochs,
                     validation_data=([padded_val_encoder_inputs, padded_val_decoder_inputs], padded_val_decoder_targets),
                     callbacks=[cp_callback, es_callback])

Epoch 1/30
6463/6464 [============================>.] - ETA: 0s - loss: 0.1900 - accuracy: 0.6730
Epoch 1: saving model to ./EngFreNMTNoAttention/training1\cp.ckpt
6464/6464 [==============================] - 679s 105ms/step - loss: 0.1900 - accuracy: 0.6730 - val_loss: 0.1294 - val_accuracy: 0.6882
Epoch 2/30
6463/6464 [============================>.] - ETA: 0s - loss: 0.1643 - accuracy: 0.7037
Epoch 2: saving model to ./EngFreNMTNoAttention/training1\cp.ckpt
6464/6464 [==============================] - 679s 105ms/step - loss: 0.1643 - accuracy: 0.7037 - val_loss: 0.1253 - val_accuracy: 0.6951
Epoch 3/30
6463/6464 [============================>.] - ETA: 0s - loss: 0.1466 - accuracy: 0.7258
Epoch 3: saving model to ./EngFreNMTNoAttention/training1\cp.ckpt
6464/6464 [==============================] - 678s 105ms/step - loss: 0.1466 - accuracy: 0.7258 - val_loss: 0.1232 - val_accuracy: 0.7001
Epoch 4/30
6463/6464 [============================>.] - ETA: 0s - loss: 0.1335 - accuracy: 0.7430

KeyboardInterrupt: 

In [70]:
import tensorflow as tf

# Reconstruct the model architecture
model = tf.keras.Model([encoder_inputs, decoder_inputs], y_proba, name='eng_fre_seq2seq_nmt_no_attention') # Define a function or code snippet to create your model architecture

# Compile the model (if needed)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Load the weights from the checkpoint file
model.load_weights(filepath="./EngFreNMTNoAttention/training1/cp.ckpt")

# Now the model is loaded with the weights from the checkpoint file and ready for inference or further training


### Saving Model and Tokenizers

In [72]:
# Saving Model
from keras.models import load_model

# Save model to HDF5 format
model.save('artifacts/Eng-French/eng_fre_seq2seq_nmt_no_attention.h5')

# Load model from HDF5 format
loaded_model = load_model('artifacts/Eng-French/eng_fre_seq2seq_nmt_no_attention.h5')


In [73]:
# loaded_model.fit([padded_train_encoder_inputs, padded_train_decoder_inputs], padded_train_decoder_outputs,
#                      batch_size=batch_size,
#                      epochs=3,
#                      validation_data=([padded_val_encoder_inputs, padded_val_decoder_inputs], padded_val_decoder_targets),
#                      callbacks=[cp_callback, es_callback])

In [74]:
# Saving Tokenizer
import io
import json


##### Save the tokenizers as JSON files. The resulting files can be downloaded by left-clicking on them.
source_tokenizer_json = source_tokenizer.to_json()
with io.open('artifacts/Eng-French/source_tokenizer.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(source_tokenizer_json, ensure_ascii=False))

target_tokenizer_json = target_tokenizer.to_json()
with io.open('artifacts/Eng-French/target_tokenizer.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(target_tokenizer_json, ensure_ascii=False))

In [75]:
# Reading Tokenizer
with io.open('artifacts/Eng-French/source_tokenizer.json', 'r', encoding='utf-8') as f:
    # Read the JSON data
    st= json.load(f)
    st = tf.keras.preprocessing.text.tokenizer_from_json(st)

### Model Evaluation

In [76]:
# Evaluate the model on the test set.
model.evaluate([padded_val_encoder_inputs, padded_val_decoder_inputs], padded_val_decoder_targets)

719/719 [==============================] - 45s 59ms/step - loss: 0.1219 - accuracy: 0.7072


[0.12190192937850952, 0.7072119116783142]

In [77]:
[layer.name for layer in model.layers]

['encoder_inputs',
 'decoder_inputs',
 'encoder_embeddings',
 'decoder_embeddings',
 'encoder_lstm',
 'decoder_lstm',
 'decoder_dense']

In [78]:
encoder_inputs = model.get_layer('encoder_inputs').input

encoder_embedding_layer = model.get_layer('encoder_embeddings')
encoder_embeddings = encoder_embedding_layer(encoder_inputs)

encoder_lstm = model.get_layer('encoder_lstm')

_, encoder_state_h, encoder_state_c = encoder_lstm(encoder_embeddings)

encoder_states = [encoder_state_h, encoder_state_c]

encoder_model_no_attention = tf.keras.Model(encoder_inputs, encoder_states)

In [79]:
decoder_inputs = model.get_layer('decoder_inputs').input

decoder_embedding_layer = model.get_layer('decoder_embeddings')
decoder_embeddings = decoder_embedding_layer(decoder_inputs)

# Inputs to represent the decoder's LSTM hidden and cell states. We'll populate 
# these manually using the encoder's output for the initial state.
decoder_input_state_h = tf.keras.Input(shape=(hidden_dim,), name='decoder_input_state_h')
decoder_input_state_c = tf.keras.Input(shape=(hidden_dim,), name='decoder_input_state_c')
decoder_input_states = [decoder_input_state_h, decoder_input_state_c]

decoder_lstm = model.get_layer('decoder_lstm')

decoder_sequence_outputs, decoder_output_state_h, decoder_output_state_c = decoder_lstm(
    decoder_embeddings, initial_state=decoder_input_states
)

# Update hidden and cell states for the next time step.
decoder_output_states = [decoder_output_state_h, decoder_output_state_c]

decoder_dense = model.get_layer('decoder_dense')
y_proba = decoder_dense(decoder_sequence_outputs)

decoder_model_no_attention = tf.keras.Model(
    [decoder_inputs] + decoder_input_states, 
    [y_proba] + decoder_output_states
) 

In [80]:
def translate_without_attention(sentence: str, 
                                source_tokenizer, encoder,
                                target_tokenizer, decoder,
                                max_translated_len = 30):

  # Vectorize the source sentence and run it through the encoder.    
  input_seq = source_tokenizer.texts_to_sequences([sentence])

  # Get the tokenized sentence to see if there are any unknown tokens.
  tokenized_sentence = source_tokenizer.sequences_to_texts(input_seq)

  states = encoder.predict(input_seq)  

  current_word = '<sos>'
  decoded_sentence = []

  while len(decoded_sentence) < max_translated_len:
    
    # Set the next input word for the decoder.
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = target_tokenizer.word_index[current_word]
    
    # Determine the next word.
    target_y_proba, h, c = decoder.predict([target_seq] + states)
    target_token_index = np.argmax(target_y_proba[0, -1, :])
    current_word = target_tokenizer.index_word[target_token_index]

    if (current_word == '<eos>'):
      break

    decoded_sentence.append(current_word)
    states = [h, c]
  
  return tokenized_sentence[0], ' '.join(decoded_sentence)


In [81]:
import random

source_sentences= source_val_sentences[600:650]
target_sentences= target_val_sentences[600:650]

In [82]:
def translate_sentences(source_sentences, target_sentences,translation_func, source_tokenizer, encoder,
                        target_tokenizer, decoder):
  translations = {'Tokenized Original': [], 'Reference': [], 'Translation': []}

  for s in source_sentences:
    tokenized_sentence, translated = translation_func(s, source_tokenizer, encoder,
                                                      target_tokenizer, decoder)

    translations['Tokenized Original'].append(tokenized_sentence)
    translations['Translation'].append(translated)

  for t in target_sentences:
      translations['Reference'].append(t)
  
  return translations

In [102]:
translate_without_attention(["he spoke"], source_tokenizer, encoder_model_no_attention, target_tokenizer, decoder_model_no_attention)

1/1 [==============================] - 0s 12ms/step


('<unk>', 'à l air')

In [84]:
translations_no_attention = pd.DataFrame(translate_sentences(source_sentences, target_sentences, translate_without_attention,
                                                             source_tokenizer, encoder_model_no_attention,
                                                             target_tokenizer, decoder_model_no_attention))
translations_no_attention

1/1 [==============================] - 0s 31ms/step


,Tokenized Original,Reference,Translation
0,good job,beau travail,bon travail
1,good job,bravo,bon travail
2,grab tom,attrape tom,emmène tom
3,grab tom,attrapez tom,emmène tom
4,grab him,attrape le,lui le prenez
5,grab him,attrapez le,lui le prenez
6,grab him,mettez lui la main dessus,lui le prenez
7,have fun,amuse toi bien,vous amusez bien
8,have fun,amusez vous bien,vous amusez bien
9,have fun,amuse toi bien,vous amusez bien


In [85]:
translations_no_attention

,Tokenized Original,Reference,Translation
0,good job,beau travail,bon travail
1,good job,bravo,bon travail
2,grab tom,attrape tom,emmène tom
3,grab tom,attrapez tom,emmène tom
4,grab him,attrape le,lui le prenez
5,grab him,attrapez le,lui le prenez
6,grab him,mettez lui la main dessus,lui le prenez
7,have fun,amuse toi bien,vous amusez bien
8,have fun,amusez vous bien,vous amusez bien
9,have fun,amuse toi bien,vous amusez bien


<!-- from tensorflow.keras.utils import plot_model
import pydot
import graphviz 

# Your model definition code goes here

# Plot the model and save the image 
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True) -->